In [1]:
!pip install transformers
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.2 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [3]:
from datasets import Dataset
import pandas as pd
from transformers import AutoTokenizer

In [6]:
csv_path = "/content/Source_Code_Plagiarism_Processed.csv"
df = pd.read_csv(csv_path)

In [7]:
label_mapping = {"Non-Plagiarized": 0, "L1": 1, "L2": 2, "L3": 3, "L4": 4, "L5": 5, "L6": 6}
df['label'] = df['PlagiarismLevel'].map(label_mapping)

dataset = Dataset.from_pandas(df)

dataset = dataset.train_test_split(test_size=0.2)
train_dataset = dataset["train"]
test_dataset = dataset["test"]

tokenizer = AutoTokenizer.from_pretrained("microsoft/unixcoder-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/938k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/444k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [8]:
# Tokenization function
def tokenize_function(examples):
    return tokenizer(
        examples['Code'],
        examples['OriginalCode'],
        truncation=True,
        padding="max_length",
        max_length=512,
    )

# Apply tokenization
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Remove unnecessary columns
train_dataset = train_dataset.remove_columns(["Code", "OriginalCode", "PlagiarismLevel"])
test_dataset = test_dataset.remove_columns(["Code", "OriginalCode", "PlagiarismLevel"])


Map:   0%|          | 0/368 [00:00<?, ? examples/s]

Map:   0%|          | 0/92 [00:00<?, ? examples/s]

In [11]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "microsoft/unixcoder-base",
    num_labels=len(label_mapping)
)

config.json:   0%|          | 0.00/691 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/504M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/unixcoder-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Training arguments
training_args = TrainingArguments(
    output_dir="./unixcoder_multiclass",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=8,
    weight_decay=0.01,
    load_best_model_at_end=True,
    logging_dir="./logs",
    logging_steps=10,
    fp16=True,  # Mixed precision training for faster performance
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [13]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer
)

<ipython-input-13-4593ec5e8e8a>:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [14]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,2.002400,1.924624
2,1.874800,1.799831
3,1.507800,1.423293
4,1.162400,1.244162
5,0.919200,1.014621
6,0.698100,0.997203
7,0.551100,0.965153
8,0.580600,0.951715


TrainOutput(global_step=368, training_loss=1.1448074579238892, metrics={'train_runtime': 243.3088, 'train_samples_per_second': 12.1, 'train_steps_per_second': 1.512, 'total_flos': 774633721036800.0, 'train_loss': 1.1448074579238892, 'epoch': 8.0})

In [17]:
output_dir = "./unixcoder_multiclass"
model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print("Fine-tuned UniXcoder model saved.")

Fine-tuned UniXcoder model saved.


In [19]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineG

In [20]:
from huggingface_hub import notebook_login

# Login to Hugging Face Hub
notebook_login()

# Push the model to the Hub
model.push_to_hub("YoussefHassan/unixcoder-multiclass-plagiarism-detector")
tokenizer.push_to_hub("YoussefHassan/unixcoder-multiclass-plagiarism-detector")

print("Model uploaded to Hugging Face Hub.")


model.safetensors:   0%|          | 0.00/504M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

Model uploaded to Hugging Face Hub.


In [22]:
from transformers import pipeline

# Load the model from Hugging Face Hub
unixcoder_pipeline = pipeline(
    "text-classification",
    model="YoussefHassan/unixcoder-multiclass-plagiarism-detector",
    tokenizer="YoussefHassan/unixcoder-multiclass-plagiarism-detector",
)

# Test Example
example_code = "def prod(c, d,k,d): return c * d-k/d"
example_original = "def sum(x, y): return x + y"

prompt = f"""
Task: Classify the level of plagiarism between the two code snippets.
Focus: Compare the coding logic, structure, and functionality. Ignore minor differences like variable names or formatting.
Output: Provide one of the following labels:
- "Non-Plagiarized": The two snippets have different functionality, even if they share structural similarities.
- "L1": The second code is almost identical to the original with minor changes like renaming variables or reformatting.
- "L2": The second code has some structural changes while preserving the logic.
- "L3": Moderate changes to logic or structure with significant similarities to the original.
- "L4": Heavy modifications with some recognizable similarity to the original.
- "L5": Significant reorganization or obfuscation, making the similarity less obvious.
- "L6": Highly transformed but still derived from the original.

Original Code:
{example_original}

Submission Code:
{example_code}
"""

# Get predictions
result = unixcoder_pipeline(prompt)
print("Prediction:", result)


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


Prediction: [{'label': 'LABEL_4', 'score': 0.2270328849554062}]
